In [0]:
# Análise para entender o comportamento da base de dados
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, when, isnan, lit
from pyspark.sql import SparkSession

# Carregando a base
df = spark.table('silver.orders_enriched')


# Verificando mais nulidades além dos 8.505 campos nulos do customer_id
# encontrei mais 2 no campo origin_platform e 1.271 no campo active -> Vou seguir com a exclusão para limpar a base
#null_counts = df_clean.select([
#    count(when(col(c).isNull(), c)).alias(c)
#    for c in df.columns
#])
#null_counts.show()

# Remoção dos nulos do campo customer_id, active e origin_platform
df_clean = df.filter(
    df["customer_id"].isNotNull() & df['active'].isNotNull() & df['origin_platform'].isNotNull()
)


# Confirmação da remoção
##total_original = df.count()
##total_limpo = df_clean.count()

##print(f"linhas removidas: {total_original - total_limpo}")

# Resumo descritivo
#df_clean.summary().show()
#df_clean.describe().show()

In [0]:
# Continuando com a análise descritiva

#df_clean.groupBy('is_target').count().orderBy("count",ascending=False).show() -- A variável target está desbalanceada (target: 2.136.745 e control: 1.525.576)
#df_clean.groupBy('active').count().orderBy("count",ascending=False).show() -- A variável true está desbalanceada (true: 3.652.134 e false: 8.914)

In [0]:
# Análise para entender o comportamento da base de dados
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import SparkSession

# Carregando a base
df = spark.table('silver.orders_time')


# Resumo descritivo
df["price_range","order_total_amount",'delivery_time','minimum_order_value'].summary().show()
#df.describe().show()

In [0]:
# Código para avaliar quantos dados são duplicados

from pyspark.sql.functions import concat_ws, count

# Carregando a base
df = spark.table('silver.orders_time')


chaves = ['order_id','customer_id','order_total_amount','origin_platform']

# Criar o concat
df_chave = df.withColumn("chave_unica", concat_ws("|",*chaves))
                                            
duplicacoes = df_chave.groupBy('chave_unica').count().filter('count > 1')
qtd_duplicacoes = duplicacoes.selectExpr("sum(count -1) as total_duplicacoes").collect()[0]['total_duplicacoes']

print(f"Quantidade de duplicatas: {qtd_duplicacoes}")
## Confirma as duplicações vistas em SQL

# VOu fazer a remoção das duplicações e criar outra base de dados sem as duplicações


In [0]:
# Análise para entender o comportamento dos dados da base distinta
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql import SparkSession

# Carregando a base
df = spark.table('silver.orders_time_distinct')


# Resumo descritivo
df["price_range","order_total_amount",'delivery_time','minimum_order_value'].summary().show()
#df.describe().show()